In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

import os
import pickle

# 画像の読み込み、train/test分離



In [2]:
from skimage import data, color, feature
import skimage.data
%run path.py

# 白黒化、縮小

# HOG計算

In [3]:
OK_name_list = os.listdir(img_path + "OK")
NG_name_list = os.listdir(img_path + "NG")

In [4]:
# 時間がかかるので、結果をpickle化してコメントアウト

'''
OK_image_data = np.vstack([ feature.hog( \
                                        rescale(skimage.io.imread(img_path + "OK/" + name ), 1.0 / 8.0, mode='constant', multichannel=True, anti_aliasing=True) \
                                       , block_norm='L1')
                           for name in OK_name_list] )

OK_image_data.shape # => (187, 222345)

NG_image_data = np.vstack([ feature.hog( \
                                        rescale(skimage.io.imread(img_path + "NG/" + name ), 1.0 / 8.0, mode='constant', multichannel=True, anti_aliasing=True) \
                                       , block_norm='L1')
                           for name in NG_name_list] )
NG_image_data.shape # => (186, 222345)


# OK_image_data, NG_image_data をpickle化して保存する

with open('OK_image_data.pickle', 'wb') as f:
    pickle.dump(OK_image_data, f)

with open('NG_image_data.pickle', 'wb') as f:
    pickle.dump(NG_image_data, f)
'''

'\nOK_image_data = np.vstack([ feature.hog(                                         rescale(skimage.io.imread(img_path + "OK/" + name ), 1.0 / 8.0, mode=\'constant\', multichannel=True, anti_aliasing=True)                                        , block_norm=\'L1\')\n                           for name in OK_name_list] )\n\nOK_image_data.shape # => (187, 222345)\n\nNG_image_data = np.vstack([ feature.hog(                                         rescale(skimage.io.imread(img_path + "NG/" + name ), 1.0 / 8.0, mode=\'constant\', multichannel=True, anti_aliasing=True)                                        , block_norm=\'L1\')\n                           for name in NG_name_list] )\nNG_image_data.shape # => (186, 222345)\n\n\n# OK_image_data, NG_image_data をpickle化して保存する\n\nwith open(\'OK_image_data.pickle\', \'wb\') as f:\n    pickle.dump(OK_image_data, f)\n\nwith open(\'NG_image_data.pickle\', \'wb\') as f:\n    pickle.dump(NG_image_data, f)\n'

In [5]:
with open('OK_image_data.pickle', 'rb') as f:
    OK_image_data = pickle.load(f)

with open('NG_image_data.pickle', 'rb') as f:
    NG_image_data = pickle.load(f)

In [6]:
from skimage.transform import rescale

img_array = skimage.io.imread(img_path + "OK/IMG_6753.JPG")
image_rescaled = rescale(img_array, 1.0 / 4.0, multichannel=True, anti_aliasing=True)

c:\program files\python37\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [7]:
orientations = 9
cells_per_block = (3, 3)
pixels_per_cell = (8, 8)

c_row, c_col = pixels_per_cell
b_row, b_col = cells_per_block
s_row, s_col = image_rescaled.shape[:2]
n_cells_row = int(s_row // c_row)
n_cells_col = int(s_col // c_col)
n_blocks_row = (n_cells_row - b_row) + 1
n_blocks_col = (n_cells_col - b_col) + 1

hog_data_size = [n_blocks_row, n_blocks_col, b_row, b_col, orientations]
print(hog_data_size)

import functools
functools.reduce(lambda x,y : x*y , hog_data_size)

[92, 124, 3, 3, 9]


924048

In [8]:
import sys
sys.getsizeof(OK_image_data)
# ->73923952

112

In [9]:
_ /  (1024 ** 2)

0.0001068115234375

In [10]:
_ * (187/10)

0.00199737548828125

In [11]:
from sklearn import  model_selection

In [12]:
target = np.zeros(len(OK_name_list) + len(NG_name_list))
target[:len(OK_name_list)] = 1

In [13]:
X_train, X_test, y_train, y_test = \
model_selection.train_test_split(np.vstack((OK_image_data, NG_image_data)), target, random_state=42 )


In [14]:
X_test.shape

(94, 222345)

In [15]:
# ナイーブベイズ

from sklearn.naive_bayes import  GaussianNB
from sklearn.model_selection import  cross_val_score

val = cross_val_score(GaussianNB(), X_train, y_train)
print(val)
print(val.mean())

c:\program files\python37\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


[0.7311828  0.87096774 0.67741935]
0.7598566308243727


In [16]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(LinearSVC(), {'C': [1.0, 2.0, 4.0, 8.0]})
grid.fit(X_train, y_train)

c:\program files\python37\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [1.0, 2.0, 4.0, 8.0]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [17]:
grid.best_score_

0.7204301075268817

In [18]:
grid.best_score_ > val.mean()
# ナイーブベイズよりもSVMのほうが悪い……

False

In [21]:
from sklearn.decomposition import PCA

pca = PCA(0.999)

compressed_data = pca.fit_transform(X_train)
compressed_data.shape


(373, 367)

In [25]:
# https://medium.com/@thimblot/data-augmentation-boost-your-image-dataset-with-few-lines-of-python-155c2dc1baec
import skimage
def random_rotation(imagearray):
    random_degree = random.uniform(-5, 5)
    return skimage.transform.rotate(imagearray, random_degree)

def 

In [26]:
X_train.shape


(279, 222345)

In [27]:
X_train[23]

array([0.09064332, 0.00349144, 0.02633925, ..., 0.00796195, 0.00707209,
       0.02335467])